Imports

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

Data preparation

In [11]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000, shuffle=False)

100%|██████████| 9.91M/9.91M [00:05<00:00, 1.75MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 55.4kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 828kB/s]
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


Simple CNN model

In [19]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(24*24*64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))  # Output shape: [64, 32, 26, 26]
        x = torch.relu(self.conv2(x))  # Output shape: [64, 64, 24, 24]
        x = torch.flatten(x, 1)        # Flattened shape: [64, 24*24*64] = [64, 36,864]
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [20]:
model = SimpleCNN()
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

Training loop (1 epoch for brevity)

In [22]:
for epoch in range(1):
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} complete")

Epoch 1 complete


Evaluation

In [24]:
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        pred = output.argmax(dim=1)
        correct += (pred == target).sum().item()
        total += target.size(0)
print(f"Test Accuracy: {correct / total:.2%}")

Test Accuracy: 98.09%


Save Model

In [30]:
torch.save(model.state_dict(), 'model.pth')